# CNN Classification with CR Dataset
<hr>

The __modus operandi__ for text classification is to use __word embedding__ for representing words and a Convolutional neural network to learn how to discriminate documents on classification problems. 

__Yoav Goldberg__ commented in _A Primer on Neural Network Models for Natural Language Processing, 2015._ :
> _The non-linearity of the network, as well as the ability to easily integrate pre-trained
word embeddings, often lead to superior classification accuracy._

He also commented in _Neural Network Methods for Natural Language Processing, 2017_ :
> ... _the CNN is in essence a feature-extracting architecture. ... . The CNNs layer's responsibility is to extract meaningful sub-structures that are useful for the overall prediction task at hand._

We will build a text classification model using CNN model on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/MPQA/MPQA.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10606, 3)


,sentence,label,split
0,complaining,0,train
1,failing to support,0,train
2,desperately needs,0,train
3,many years of decay,0,train
4,no quick fix,0,train
...,...,...,...
10601,urged,1,train
10602,strictly abide,1,train
10603,hope,1,train
10604,strictly abide,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10606 entries, 0 to 10605
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10606 non-null  object
 1   label     10606 non-null  int32 
 2   split     10606 non-null  object
dtypes: int32(1), object(2)
memory usage: 207.3+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,7294,7294
1,3312,3312


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'complaining'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  no quick fix
Into a sequence of int: [25, 945, 1476]
Into a padded sequence: [  25  945 1476    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
of 3
to 4
a 5
and 6
not 7
is 8
in 9
be 10
6236


# Model 1: Embedding Random
<hr>

A __standard model__ for document classification is to use (quoted from __Jason Brownlee__, the author of [machinelearningmastery.com](https://machinelearningmastery.com)):
>- Word Embedding: A distributed representation of words where different words that have a similar meaning (based on their usage) also have a similar representation.
>- Convolutional Model: A feature extraction model that learns to extract salient features from documents represented using a word embedding.
>- Fully Connected Model: The interpretation of extracted features in terms of a predictive output.


Therefore, the model is comprised of the following elements:
- __Input layer__ that defines the length of input sequences.
- __Embedding layer__ set to the size of the vocabulary and 100-dimensional real-valued representations.
- __Conv1D layer__ with 32 filters and a kernel size set to the number of words to read at once.
- __MaxPooling1D layer__ to consolidate the output from the convolutional layer.
- __Flatten layer__ to reduce the three-dimensional output to two dimensional for concatenation.

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

We will perform the best parameter using __grid search__ and 10-fold cross validation.

## CNN-GRU Model

Now, we will build Convolutional Neural Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from `CNN for Sentence Classification` by *Yoon Kim*.

Now, we will define our CNN model as follows:
- One Conv layer with 100 filters, kernel size 5, and relu activation function;
- One MaxPool layer with pool size = 2;
- One Dropout layer after flattened;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [11]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dropout, MaxPool1D, Flatten, Dense, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate

In [12]:
def define_model(filters = 100, kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length, mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [13]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 98, 100)      90100       embedding[0][0]                  
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 49, 100)      0           conv1d[0][0]                     
______________________________________________________________________________________________

In [14]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=7, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [15]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            # Define the input shape
            model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)
                    
                else:
                    print('Done!')

            # evaluate the model
            loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

Epoch 1/100
191/191 [==============================] - 46s 148ms/step - loss: 0.6530 - accuracy: 0.6011 - val_loss: 0.5630 - val_accuracy: 0.6739
Epoch 2/100
191/191 [==============================] - 23s 118ms/step - loss: 0.5231 - accuracy: 0.6950 - val_loss: 0.5324 - val_accuracy: 0.6739
Epoch 3/100
191/191 [==============================] - 22s 117ms/step - loss: 0.4822 - accuracy: 0.6999 - val_loss: 0.5169 - val_accuracy: 0.6739
Epoch 4/100
191/191 [==============================] - 23s 119ms/step - loss: 0.4586 - accuracy: 0.6922 - val_loss: 0.5023 - val_accuracy: 0.6739
Epoch 5/100
191/191 [==============================] - 22s 116ms/step - loss: 0.4341 - accuracy: 0.6937 - val_loss: 0.4956 - val_accuracy: 0.6739
Epoch 6/100
191/191 [==============================] - 23s 119ms/step - loss: 0.4126 - accuracy: 0.6993 - val_loss: 0.4808 - val_accuracy: 0.8549
Epoch 7/100
191/191 [==============================] - 23s 119ms/step - loss: 0.3948 - accuracy: 0.9438 - val_loss: 0.4705 -

Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 88.12441229820251
Done!
Test Accuracy: 88.12441229820251
Epoch 1/100
191/191 [==============================] - 52s 189ms/step - loss: 0.6384 - accuracy: 0.6884 - val_loss: 0.5763 - val_accuracy: 0.7135
Epoch 2/100
191/191 [==============================] - 32s 170ms/step - loss: 0.5711 - accuracy: 0.6854 - val_loss: 0.5436 - val_accuracy: 0.7135
Epoch 3/100
191/191 [==============================] - 31s 162ms/step - loss: 0.5316 - accuracy: 0.6930 - val_loss: 0.5172 - val_accuracy: 0.7135
Epoch 4/100
191/191 [==============================] - 30s 160ms/step - loss: 0.4900 - accuracy: 0.7048 - val_loss: 0.4964 - val_accuracy: 0.7135
Epoch 5/100
191/191 [==============================] - 31s 161ms/step - loss: 0.4682 - accuracy: 0.7194 - val_loss: 0.4757 - val_accuracy: 0.8662
Epoch 6/100
191/191 [==============================] - 30s 157ms/step - loss: 0.4356 - accuracy: 0.9471 - val_loss

Epoch 15/100
191/191 [==============================] - 34s 177ms/step - loss: 0.2284 - accuracy: 0.9553 - val_loss: 0.3687 - val_accuracy: 0.8387
Epoch 16/100
191/191 [==============================] - 33s 175ms/step - loss: 0.2103 - accuracy: 0.9595 - val_loss: 0.3650 - val_accuracy: 0.8387
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Test Accuracy: 87.35849261283875
Done!
Test Accuracy: 87.35849261283875
Epoch 1/100
191/191 [==============================] - 68s 268ms/step - loss: 0.7886 - accuracy: 0.4488 - val_loss: 0.6245 - val_accuracy: 0.7953
Epoch 2/100
191/191 [==============================] - 35s 183ms/step - loss: 0.5906 - accuracy: 0.8338 - val_loss: 0.5716 - val_accuracy: 0.8283
Epoch 3/100
191/191 [==============================] - 34s 178ms/step - loss: 0.5295 - accuracy: 0.8686 - val_loss: 0.5343 - val_accuracy: 0.8670
Epoch 4/100
191/191 [==============================] - 34s 180ms/step - loss: 0.4863 - accuracy: 0.9288 - val_lo

191/191 [==============================] - 49s 258ms/step - loss: 0.4627 - accuracy: 0.6975 - val_loss: 0.4908 - val_accuracy: 0.6795
Epoch 3/100
191/191 [==============================] - 51s 269ms/step - loss: 0.4258 - accuracy: 0.6958 - val_loss: 0.4743 - val_accuracy: 0.6795
Epoch 4/100
191/191 [==============================] - 49s 256ms/step - loss: 0.3930 - accuracy: 0.8662 - val_loss: 0.4619 - val_accuracy: 0.8567
Epoch 5/100
191/191 [==============================] - 51s 268ms/step - loss: 0.3763 - accuracy: 0.9455 - val_loss: 0.4448 - val_accuracy: 0.8520
Epoch 6/100
191/191 [==============================] - 51s 265ms/step - loss: 0.3150 - accuracy: 0.9524 - val_loss: 0.3827 - val_accuracy: 0.8549
Epoch 7/100
191/191 [==============================] - 49s 258ms/step - loss: 0.2451 - accuracy: 0.9509 - val_loss: 0.3793 - val_accuracy: 0.8558
Epoch 8/100
191/191 [==============================] - 49s 255ms/step - loss: 0.2136 - accuracy: 0.9504 - val_loss: 0.3868 - val_accurac

Epoch 7/100
191/191 [==============================] - 46s 239ms/step - loss: 0.2550 - accuracy: 0.9415 - val_loss: 0.3967 - val_accuracy: 0.8313
Epoch 8/100
191/191 [==============================] - 46s 240ms/step - loss: 0.2356 - accuracy: 0.9429 - val_loss: 0.3967 - val_accuracy: 0.8247
Epoch 9/100
191/191 [==============================] - 46s 240ms/step - loss: 0.2241 - accuracy: 0.9467 - val_loss: 0.4022 - val_accuracy: 0.8247
Epoch 10/100
191/191 [==============================] - 46s 239ms/step - loss: 0.2083 - accuracy: 0.9510 - val_loss: 0.4054 - val_accuracy: 0.8256
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 86.80490255355835
Done!
Test Accuracy: 86.80490255355835
Epoch 1/100
191/191 [==============================] - 55s 201ms/step - loss: 0.7552 - accuracy: 0.4856 - val_loss: 0.5960 - val_accuracy: 0.8123
Epoch 2/100
191/191 [==============================] - 26s 138ms/step - loss: 0.5625 - accuracy: 0.8933 - val_los

...

191/191 [==============================] - 54s 281ms/step - loss: 0.5626 - accuracy: 0.8927 - val_loss: 0.5596 - val_accuracy: 0.8407
Epoch 3/100
191/191 [==============================] - 53s 279ms/step - loss: 0.5036 - accuracy: 0.9202 - val_loss: 0.5347 - val_accuracy: 0.8379
Epoch 4/100
191/191 [==============================] - 53s 276ms/step - loss: 0.4586 - accuracy: 0.9287 - val_loss: 0.5019 - val_accuracy: 0.8492
Epoch 5/100
191/191 [==============================] - 53s 276ms/step - loss: 0.4242 - accuracy: 0.9348 - val_loss: 0.4761 - val_accuracy: 0.8558
Epoch 6/100
191/191 [==============================] - 53s 276ms/step - loss: 0.3907 - accuracy: 0.9439 - val_loss: 0.4625 - val_accuracy: 0.8483
Epoch 7/100
191/191 [==============================] - 53s 277ms/step - loss: 0.3637 - accuracy: 0.9467 - val_loss: 0.4579 - val_accuracy: 0.8407
Epoch 8/100
191/191 [==============================] - 52s 270ms/step - loss: 0.3371 - accuracy: 0.9512 - val_loss: 0.4457 - val_accurac

Epoch 1/100
191/191 [==============================] - 73s 297ms/step - loss: 0.5901 - accuracy: 0.6904 - val_loss: 0.4983 - val_accuracy: 0.6925
Epoch 2/100
191/191 [==============================] - 41s 214ms/step - loss: 0.4298 - accuracy: 0.7224 - val_loss: 0.4572 - val_accuracy: 0.8358
Epoch 3/100
191/191 [==============================] - 41s 214ms/step - loss: 0.3616 - accuracy: 0.8205 - val_loss: 0.4341 - val_accuracy: 0.8453
Epoch 4/100
191/191 [==============================] - 42s 217ms/step - loss: 0.3240 - accuracy: 0.9434 - val_loss: 0.4220 - val_accuracy: 0.8425
Epoch 5/100
191/191 [==============================] - 40s 208ms/step - loss: 0.2914 - accuracy: 0.9542 - val_loss: 0.4327 - val_accuracy: 0.7972
Epoch 6/100
191/191 [==============================] - 40s 211ms/step - loss: 0.2709 - accuracy: 0.9563 - val_loss: 0.4230 - val_accuracy: 0.7981
Epoch 7/100
191/191 [==============================] - 39s 207ms/step - loss: 0.2430 - accuracy: 0.9596 - val_loss: 0.4204 -

Epoch 15/100
191/191 [==============================] - 47s 248ms/step - loss: 0.1475 - accuracy: 0.9673 - val_loss: 0.3968 - val_accuracy: 0.8538
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
Test Accuracy: 86.32075190544128
Done!
Test Accuracy: 86.32075190544128

  Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  85.485393  85.862392  84.637135  88.124412  87.087655   
1       relu       2  85.579640  85.673892  86.427897  85.579640  86.145145   
2       relu       3  87.558907  85.579640  87.370408  85.391140  86.239398   
3       relu       4  85.862392  85.862392  85.202640  85.862392  86.239398   
4       relu       5  87.464654  86.145145  85.579640  86.050898  85.108387   

        acc6       acc7       acc8       acc9      acc10        AVG  
0  85.485393  86.132073  87.358493  86.698115  86.509436  86.338050  
1  86.804903  86.037737  85.566038  85.943395  85.943395  85.970168  
2  86.71065

191/191 [==============================] - 46s 239ms/step - loss: 0.5359 - accuracy: 0.6890 - val_loss: 0.5367 - val_accuracy: 0.6833
Epoch 3/100
191/191 [==============================] - 46s 243ms/step - loss: 0.5002 - accuracy: 0.6820 - val_loss: 0.5131 - val_accuracy: 0.6833
Epoch 4/100
191/191 [==============================] - 45s 237ms/step - loss: 0.4669 - accuracy: 0.6867 - val_loss: 0.4937 - val_accuracy: 0.6833
Epoch 5/100
191/191 [==============================] - 46s 242ms/step - loss: 0.4378 - accuracy: 0.9028 - val_loss: 0.4866 - val_accuracy: 0.8417
Epoch 6/100
191/191 [==============================] - 46s 240ms/step - loss: 0.4077 - accuracy: 0.9484 - val_loss: 0.4663 - val_accuracy: 0.8549
Epoch 7/100
191/191 [==============================] - 46s 241ms/step - loss: 0.3895 - accuracy: 0.9448 - val_loss: 0.4574 - val_accuracy: 0.8483
Epoch 8/100
191/191 [==============================] - 46s 239ms/step - loss: 0.3665 - accuracy: 0.9498 - val_loss: 0.4533 - val_accurac

Epoch 4/100
191/191 [==============================] - 47s 246ms/step - loss: 0.2968 - accuracy: 0.9458 - val_loss: 0.4137 - val_accuracy: 0.8425
Epoch 5/100
191/191 [==============================] - 46s 242ms/step - loss: 0.2648 - accuracy: 0.9492 - val_loss: 0.4114 - val_accuracy: 0.8189
Epoch 6/100
191/191 [==============================] - 46s 241ms/step - loss: 0.2363 - accuracy: 0.9585 - val_loss: 0.4056 - val_accuracy: 0.8142
Epoch 7/100
191/191 [==============================] - 47s 245ms/step - loss: 0.2151 - accuracy: 0.9590 - val_loss: 0.4066 - val_accuracy: 0.8170
Epoch 8/100
191/191 [==============================] - 47s 245ms/step - loss: 0.2002 - accuracy: 0.9576 - val_loss: 0.4147 - val_accuracy: 0.8160
Epoch 9/100
191/191 [==============================] - 46s 241ms/step - loss: 0.1886 - accuracy: 0.9603 - val_loss: 0.4163 - val_accuracy: 0.8113
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 85.00000238418579
Done!
T

## Summary

In [16]:
record.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,relu,1,85.485393,85.862392,84.637135,88.124412,87.087655,85.485393,86.132073,87.358493,86.698115,86.509436,86.338050
2,relu,3,87.558907,85.579640,87.370408,85.391140,86.239398,86.710650,85.094339,84.528303,87.735850,86.509436,86.271807
3,relu,4,85.862392,85.862392,85.202640,85.862392,86.239398,87.464654,86.415094,86.603773,87.075472,85.377359,86.196557
1,relu,2,85.579640,85.673892,86.427897,85.579640,86.145145,86.804903,86.037737,85.566038,85.943395,85.943395,85.970168
4,relu,5,87.464654,86.145145,85.579640,86.050898,85.108387,86.899149,84.528303,84.622639,85.660380,86.320752,85.837995
5,relu,6,85.108387,85.862392,85.014135,85.485393,86.333650,85.956645,86.981130,85.000002,85.000002,86.037737,85.677947


In [17]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,86.33805


In [18]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_MPQA.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [19]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [20]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [21]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [22]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 6083 words present from 6236 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [23]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()
print('emb_mean: ', emb_mean)
print('emb_std: ', emb_std)

emb_mean:  -0.003527845
emb_std:  0.13315111


In [24]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [25]:
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## MLP-CNN-GRU Model

In [26]:
def define_model_2(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [27]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_64"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_129 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_128 (Embedding)       (None, 100, 300)     300000      input_129[0][0]                  
__________________________________________________________________________________________________
conv1d_64 (Conv1D)              (None, 98, 100)      90100       embedding_128[0][0]              
__________________________________________________________________________________________________
max_pooling1d_64 (MaxPooling1D) (None, 49, 100)      0           conv1d_64[0][0]                  
___________________________________________________________________________________________

## Train and Test the Model

In [28]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=7, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [29]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                    
                else:
                    print('Done!')

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

Epoch 1/100
191/191 [==============================] - 43s 151ms/step - loss: 0.6958 - accuracy: 0.5759 - val_loss: 0.5936 - val_accuracy: 0.8501
Epoch 2/100
191/191 [==============================] - 17s 91ms/step - loss: 0.5684 - accuracy: 0.8816 - val_loss: 0.5405 - val_accuracy: 0.8850
Epoch 3/100
191/191 [==============================] - 20s 102ms/step - loss: 0.5178 - accuracy: 0.8965 - val_loss: 0.5040 - val_accuracy: 0.8841
Epoch 4/100
191/191 [==============================] - 25s 130ms/step - loss: 0.4846 - accuracy: 0.8961 - val_loss: 0.4716 - val_accuracy: 0.8897
Epoch 5/100
191/191 [==============================] - 26s 135ms/step - loss: 0.4471 - accuracy: 0.9062 - val_loss: 0.4457 - val_accuracy: 0.8897
Epoch 6/100
191/191 [==============================] - 25s 133ms/step - loss: 0.4165 - accuracy: 0.9148 - val_loss: 0.4288 - val_accuracy: 0.8860
Epoch 7/100
191/191 [==============================] - 25s 131ms/step - loss: 0.3999 - accuracy: 0.9105 - val_loss: 0.4067 - 

191/191 [==============================] - 17s 91ms/step - loss: 0.1588 - accuracy: 0.9546 - val_loss: 0.3260 - val_accuracy: 0.8935
Epoch 30/100
191/191 [==============================] - 17s 90ms/step - loss: 0.1644 - accuracy: 0.9524 - val_loss: 0.3212 - val_accuracy: 0.8916
Epoch 31/100
191/191 [==============================] - 17s 88ms/step - loss: 0.1620 - accuracy: 0.9549 - val_loss: 0.3279 - val_accuracy: 0.8907
Epoch 32/100
191/191 [==============================] - 17s 91ms/step - loss: 0.1552 - accuracy: 0.9534 - val_loss: 0.3326 - val_accuracy: 0.8897
Restoring model weights from the end of the best epoch.
Epoch 00032: early stopping
Test Accuracy: 89.72667455673218
Done!
Epoch 1/100
191/191 [==============================] - 34s 103ms/step - loss: 0.5449 - accuracy: 0.6957 - val_loss: 0.4293 - val_accuracy: 0.8360
Epoch 2/100
191/191 [==============================] - 12s 63ms/step - loss: 0.4139 - accuracy: 0.8375 - val_loss: 0.3989 - val_accuracy: 0.8671
Epoch 3/100
191

191/191 [==============================] - 51s 186ms/step - loss: 0.6992 - accuracy: 0.5896 - val_loss: 0.6494 - val_accuracy: 0.8473
Epoch 2/100
191/191 [==============================] - 31s 160ms/step - loss: 0.6362 - accuracy: 0.8692 - val_loss: 0.6047 - val_accuracy: 0.8822
Epoch 3/100
191/191 [==============================] - 31s 161ms/step - loss: 0.5915 - accuracy: 0.8875 - val_loss: 0.5667 - val_accuracy: 0.8822
Epoch 4/100
191/191 [==============================] - 30s 155ms/step - loss: 0.5526 - accuracy: 0.9005 - val_loss: 0.5363 - val_accuracy: 0.8860
Epoch 5/100
191/191 [==============================] - 30s 158ms/step - loss: 0.5196 - accuracy: 0.9057 - val_loss: 0.5100 - val_accuracy: 0.8822
Epoch 6/100
191/191 [==============================] - 30s 158ms/step - loss: 0.4927 - accuracy: 0.9055 - val_loss: 0.4870 - val_accuracy: 0.8831
Epoch 7/100
191/191 [==============================] - 30s 156ms/step - loss: 0.4666 - accuracy: 0.9058 - val_loss: 0.4671 - val_accurac

191/191 [==============================] - 49s 256ms/step - loss: 0.4357 - accuracy: 0.9053 - val_loss: 0.4523 - val_accuracy: 0.8651
Epoch 5/100
191/191 [==============================] - 51s 266ms/step - loss: 0.4008 - accuracy: 0.9129 - val_loss: 0.4335 - val_accuracy: 0.8689
Epoch 6/100
191/191 [==============================] - 50s 261ms/step - loss: 0.3731 - accuracy: 0.9179 - val_loss: 0.4257 - val_accuracy: 0.8679
Epoch 7/100
191/191 [==============================] - 50s 264ms/step - loss: 0.3654 - accuracy: 0.9116 - val_loss: 0.4145 - val_accuracy: 0.8679
Epoch 8/100
191/191 [==============================] - 49s 257ms/step - loss: 0.3457 - accuracy: 0.9192 - val_loss: 0.4149 - val_accuracy: 0.8632
Epoch 9/100
191/191 [==============================] - 50s 261ms/step - loss: 0.3296 - accuracy: 0.9221 - val_loss: 0.4011 - val_accuracy: 0.8670
Epoch 10/100
191/191 [==============================] - 49s 257ms/step - loss: 0.3124 - accuracy: 0.9276 - val_loss: 0.3984 - val_accura

191/191 [==============================] - 27s 142ms/step - loss: 0.4757 - accuracy: 0.9014 - val_loss: 0.4736 - val_accuracy: 0.8831
Epoch 6/100
191/191 [==============================] - 28s 145ms/step - loss: 0.4470 - accuracy: 0.9073 - val_loss: 0.4539 - val_accuracy: 0.8794
Epoch 7/100
191/191 [==============================] - 27s 142ms/step - loss: 0.4199 - accuracy: 0.9145 - val_loss: 0.4332 - val_accuracy: 0.8822
Epoch 8/100
191/191 [==============================] - 34s 179ms/step - loss: 0.4012 - accuracy: 0.9121 - val_loss: 0.4247 - val_accuracy: 0.8756
Epoch 9/100
191/191 [==============================] - 61s 321ms/step - loss: 0.3850 - accuracy: 0.9138 - val_loss: 0.4047 - val_accuracy: 0.8860
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 88.59566450119019
Done!
Epoch 1/100
191/191 [==============================] - 87s 362ms/step - loss: 0.6202 - accuracy: 0.6852 - val_loss: 0.5873 - val_accuracy: 0.6795
Epoch 2/100
1

...

191/191 [==============================] - 46s 243ms/step - loss: 0.3778 - accuracy: 0.9126 - val_loss: 0.3928 - val_accuracy: 0.8869
Epoch 7/100
191/191 [==============================] - 46s 243ms/step - loss: 0.3634 - accuracy: 0.9151 - val_loss: 0.3795 - val_accuracy: 0.8926
Epoch 8/100
191/191 [==============================] - 47s 246ms/step - loss: 0.3556 - accuracy: 0.9150 - val_loss: 0.3743 - val_accuracy: 0.8944
Epoch 9/100
191/191 [==============================] - 47s 246ms/step - loss: 0.3387 - accuracy: 0.9178 - val_loss: 0.3663 - val_accuracy: 0.8878
Epoch 10/100
191/191 [==============================] - 46s 243ms/step - loss: 0.3308 - accuracy: 0.9180 - val_loss: 0.3549 - val_accuracy: 0.8963
Epoch 11/100
191/191 [==============================] - 47s 247ms/step - loss: 0.3089 - accuracy: 0.9284 - val_loss: 0.3580 - val_accuracy: 0.8897
Epoch 12/100
191/191 [==============================] - 47s 245ms/step - loss: 0.3057 - accuracy: 0.9272 - val_loss: 0.3543 - val_accu

191/191 [==============================] - 40s 211ms/step - loss: 0.3410 - accuracy: 0.9203 - val_loss: 0.3869 - val_accuracy: 0.8792
Epoch 9/100
191/191 [==============================] - 40s 211ms/step - loss: 0.3342 - accuracy: 0.9164 - val_loss: 0.3817 - val_accuracy: 0.8708
Epoch 10/100
191/191 [==============================] - 40s 212ms/step - loss: 0.3174 - accuracy: 0.9242 - val_loss: 0.3791 - val_accuracy: 0.8717
Epoch 11/100
191/191 [==============================] - 41s 215ms/step - loss: 0.3089 - accuracy: 0.9250 - val_loss: 0.3695 - val_accuracy: 0.8726
Epoch 12/100
191/191 [==============================] - 42s 220ms/step - loss: 0.3001 - accuracy: 0.9265 - val_loss: 0.3679 - val_accuracy: 0.8708
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 88.96226286888123
Done!
Epoch 1/100
191/191 [==============================] - 73s 286ms/step - loss: 0.7201 - accuracy: 0.5466 - val_loss: 0.5909 - val_accuracy: 0.8283
Epoch 2/10

191/191 [==============================] - 51s 264ms/step - loss: 0.3146 - accuracy: 0.9227 - val_loss: 0.3641 - val_accuracy: 0.8926
Epoch 12/100
191/191 [==============================] - 50s 261ms/step - loss: 0.3015 - accuracy: 0.9253 - val_loss: 0.3620 - val_accuracy: 0.8869
Epoch 13/100
191/191 [==============================] - 50s 263ms/step - loss: 0.2868 - accuracy: 0.9320 - val_loss: 0.3519 - val_accuracy: 0.8973
Epoch 14/100
191/191 [==============================] - 50s 262ms/step - loss: 0.2900 - accuracy: 0.9250 - val_loss: 0.3482 - val_accuracy: 0.8907
Epoch 15/100
191/191 [==============================] - 47s 246ms/step - loss: 0.2795 - accuracy: 0.9282 - val_loss: 0.3548 - val_accuracy: 0.8907
Epoch 16/100
191/191 [==============================] - 46s 243ms/step - loss: 0.2722 - accuracy: 0.9300 - val_loss: 0.3428 - val_accuracy: 0.8935
Epoch 17/100
191/191 [==============================] - 46s 242ms/step - loss: 0.2609 - accuracy: 0.9331 - val_loss: 0.3343 - val_a

191/191 [==============================] - 105s 551ms/step - loss: 0.2925 - accuracy: 0.9278 - val_loss: 0.3894 - val_accuracy: 0.8671
Epoch 15/100
191/191 [==============================] - 107s 559ms/step - loss: 0.2942 - accuracy: 0.9234 - val_loss: 0.3831 - val_accuracy: 0.8615
Epoch 16/100
191/191 [==============================] - 107s 558ms/step - loss: 0.2838 - accuracy: 0.9274 - val_loss: 0.3815 - val_accuracy: 0.8643
Epoch 17/100
191/191 [==============================] - 106s 554ms/step - loss: 0.2750 - accuracy: 0.9291 - val_loss: 0.3816 - val_accuracy: 0.8652
Epoch 18/100
191/191 [==============================] - 106s 553ms/step - loss: 0.2809 - accuracy: 0.9243 - val_loss: 0.3789 - val_accuracy: 0.8671
Epoch 19/100
191/191 [==============================] - 107s 558ms/step - loss: 0.2476 - accuracy: 0.9374 - val_loss: 0.3883 - val_accuracy: 0.8633
Epoch 20/100
191/191 [==============================] - 105s 547ms/step - loss: 0.2539 - accuracy: 0.9311 - val_loss: 0.3813 

191/191 [==============================] - 90s 473ms/step - loss: 0.2347 - accuracy: 0.9408 - val_loss: 0.3873 - val_accuracy: 0.8651
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
Test Accuracy: 86.98112964630127
Done!
Epoch 1/100
191/191 [==============================] - 114s 503ms/step - loss: 0.6255 - accuracy: 0.6799 - val_loss: 0.5536 - val_accuracy: 0.7019
Epoch 2/100
191/191 [==============================] - 102s 532ms/step - loss: 0.5451 - accuracy: 0.7165 - val_loss: 0.5195 - val_accuracy: 0.7019
Epoch 3/100
191/191 [==============================] - 101s 530ms/step - loss: 0.5037 - accuracy: 0.7193 - val_loss: 0.4980 - val_accuracy: 0.7019
Epoch 4/100
191/191 [==============================] - 101s 527ms/step - loss: 0.4842 - accuracy: 0.7340 - val_loss: 0.4737 - val_accuracy: 0.8774
Epoch 5/100
191/191 [==============================] - 101s 531ms/step - loss: 0.4554 - accuracy: 0.9009 - val_loss: 0.4592 - val_accuracy: 0.8802
Epoch 6/

## Summary

In [30]:
record2.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,relu,1,89.349669,88.784164,89.726675,89.066917,87.653160,89.443922,89.056605,87.830186,87.264150,85.943395,88.411884
2,relu,3,87.935907,88.878417,87.464654,88.689917,89.915174,88.784164,87.830186,86.792451,89.056605,87.924528,88.327200
3,relu,4,87.558907,87.558907,88.689917,86.899149,88.501412,88.689917,90.094340,88.396227,87.452829,88.396227,88.223783
7,relu,8,87.841660,89.726675,87.935907,89.066917,87.370408,88.972670,86.981130,88.301885,87.830186,87.547171,88.157461
1,relu,2,87.935907,88.595665,88.501412,88.595665,87.464654,87.087655,88.773584,88.301885,87.547171,88.679248,88.148285
5,relu,6,88.595665,87.747407,88.218659,88.218659,88.689917,88.312912,89.528304,86.320752,87.358493,87.735850,88.072662
4,relu,5,85.202640,86.710650,88.595665,88.030159,85.862392,89.443922,88.679248,88.301885,89.245284,89.716983,87.978883
6,relu,7,86.616397,90.103674,87.181902,86.899149,86.616397,89.632422,86.132073,88.301885,88.962263,89.245284,87.969145


In [31]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,88.411884


In [32]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_MPQA_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## MLP-CNN-GRU Model

In [35]:
def define_model_3(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [36]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_148"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_297 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_296 (Embedding)       (None, 100, 300)     300000      input_297[0][0]                  
__________________________________________________________________________________________________
conv1d_148 (Conv1D)             (None, 98, 100)      90100       embedding_296[0][0]              
__________________________________________________________________________________________________
max_pooling1d_148 (MaxPooling1D (None, 49, 100)      0           conv1d_148[0][0]                 
__________________________________________________________________________________________

## Train and Test the Model

In [37]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=7, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [38]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                    
                else:
                    print('Done!')
            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

Epoch 1/100
191/191 [==============================] - 143s 608ms/step - loss: 0.6896 - accuracy: 0.5803 - val_loss: 0.6019 - val_accuracy: 0.8379
Epoch 2/100
191/191 [==============================] - 109s 570ms/step - loss: 0.5650 - accuracy: 0.9182 - val_loss: 0.5563 - val_accuracy: 0.8680
Epoch 3/100
191/191 [==============================] - 110s 576ms/step - loss: 0.5034 - accuracy: 0.9408 - val_loss: 0.5204 - val_accuracy: 0.8699
Epoch 4/100
191/191 [==============================] - 107s 563ms/step - loss: 0.4399 - accuracy: 0.9455 - val_loss: 0.4534 - val_accuracy: 0.8671
Epoch 5/100
191/191 [==============================] - 98s 513ms/step - loss: 0.3576 - accuracy: 0.9554 - val_loss: 0.3914 - val_accuracy: 0.8709
Epoch 6/100
191/191 [==============================] - 97s 506ms/step - loss: 0.2978 - accuracy: 0.9546 - val_loss: 0.3737 - val_accuracy: 0.8671
Epoch 7/100
191/191 [==============================] - 96s 501ms/step - loss: 0.2604 - accuracy: 0.9566 - val_loss: 0.36

191/191 [==============================] - 100s 523ms/step - loss: 0.2885 - accuracy: 0.9581 - val_loss: 0.3865 - val_accuracy: 0.8671
Epoch 7/100
191/191 [==============================] - 105s 548ms/step - loss: 0.2663 - accuracy: 0.9589 - val_loss: 0.3801 - val_accuracy: 0.8699
Epoch 8/100
191/191 [==============================] - 97s 509ms/step - loss: 0.2428 - accuracy: 0.9620 - val_loss: 0.3726 - val_accuracy: 0.8690
Epoch 9/100
191/191 [==============================] - 97s 508ms/step - loss: 0.2317 - accuracy: 0.9603 - val_loss: 0.3722 - val_accuracy: 0.8680
Epoch 10/100
191/191 [==============================] - 97s 508ms/step - loss: 0.2171 - accuracy: 0.9623 - val_loss: 0.3656 - val_accuracy: 0.8680
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 87.27615475654602
Done!
Epoch 1/100
191/191 [==============================] - 111s 491ms/step - loss: 0.5571 - accuracy: 0.6936 - val_loss: 0.4842 - val_accuracy: 0.6767
Epoch 2/1

Epoch 1/100
191/191 [==============================] - 91s 379ms/step - loss: 0.6515 - accuracy: 0.7102 - val_loss: 0.5580 - val_accuracy: 0.9019
Epoch 2/100
191/191 [==============================] - 57s 297ms/step - loss: 0.5401 - accuracy: 0.9243 - val_loss: 0.5101 - val_accuracy: 0.8981
Epoch 3/100
191/191 [==============================] - 57s 297ms/step - loss: 0.4866 - accuracy: 0.9392 - val_loss: 0.4800 - val_accuracy: 0.8896
Epoch 4/100
191/191 [==============================] - 56s 296ms/step - loss: 0.4509 - accuracy: 0.9452 - val_loss: 0.4566 - val_accuracy: 0.8896
Epoch 5/100
191/191 [==============================] - 57s 297ms/step - loss: 0.4215 - accuracy: 0.9473 - val_loss: 0.4331 - val_accuracy: 0.8953
Epoch 6/100
191/191 [==============================] - 57s 296ms/step - loss: 0.3903 - accuracy: 0.9555 - val_loss: 0.4170 - val_accuracy: 0.8896
Epoch 7/100
191/191 [==============================] - 57s 297ms/step - loss: 0.3646 - accuracy: 0.9615 - val_loss: 0.4041 -

191/191 [==============================] - 55s 290ms/step - loss: 0.3288 - accuracy: 0.9593 - val_loss: 0.4181 - val_accuracy: 0.8567
Epoch 7/100
191/191 [==============================] - 55s 290ms/step - loss: 0.3087 - accuracy: 0.9619 - val_loss: 0.4057 - val_accuracy: 0.8577
Epoch 8/100
191/191 [==============================] - 56s 291ms/step - loss: 0.2853 - accuracy: 0.9651 - val_loss: 0.4009 - val_accuracy: 0.8567
Epoch 9/100
191/191 [==============================] - 56s 291ms/step - loss: 0.2738 - accuracy: 0.9647 - val_loss: 0.3988 - val_accuracy: 0.8549
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 87.37040758132935
Done!
Epoch 1/100
191/191 [==============================] - 69s 284ms/step - loss: 0.7128 - accuracy: 0.6061 - val_loss: 0.6161 - val_accuracy: 0.8332
Epoch 2/100
191/191 [==============================] - 52s 273ms/step - loss: 0.5792 - accuracy: 0.9110 - val_loss: 0.5655 - val_accuracy: 0.8680
Epoch 3/100
1

Epoch 9/100
191/191 [==============================] - 57s 297ms/step - loss: 0.3897 - accuracy: 0.9589 - val_loss: 0.4412 - val_accuracy: 0.8726
Epoch 10/100
191/191 [==============================] - 57s 298ms/step - loss: 0.3716 - accuracy: 0.9573 - val_loss: 0.4328 - val_accuracy: 0.8660
Epoch 11/100
191/191 [==============================] - 57s 297ms/step - loss: 0.3516 - accuracy: 0.9597 - val_loss: 0.4221 - val_accuracy: 0.8670
Epoch 12/100
191/191 [==============================] - 57s 297ms/step - loss: 0.3367 - accuracy: 0.9598 - val_loss: 0.4145 - val_accuracy: 0.8670
Epoch 13/100
191/191 [==============================] - 57s 300ms/step - loss: 0.3207 - accuracy: 0.9589 - val_loss: 0.4125 - val_accuracy: 0.8585
Epoch 14/100
191/191 [==============================] - 57s 301ms/step - loss: 0.3072 - accuracy: 0.9601 - val_loss: 0.4028 - val_accuracy: 0.8623
Epoch 15/100
191/191 [==============================] - 57s 300ms/step - loss: 0.2939 - accuracy: 0.9614 - val_loss: 0.

...

191/191 [==============================] - 59s 307ms/step - loss: 0.2730 - accuracy: 0.9445 - val_loss: 0.3384 - val_accuracy: 0.9010
Epoch 8/100
191/191 [==============================] - 60s 313ms/step - loss: 0.2537 - accuracy: 0.9613 - val_loss: 0.3356 - val_accuracy: 0.8831
Epoch 9/100
191/191 [==============================] - 60s 312ms/step - loss: 0.2391 - accuracy: 0.9602 - val_loss: 0.3254 - val_accuracy: 0.8888
Epoch 10/100
191/191 [==============================] - 59s 311ms/step - loss: 0.2182 - accuracy: 0.9639 - val_loss: 0.3195 - val_accuracy: 0.8944
Epoch 11/100
191/191 [==============================] - 59s 311ms/step - loss: 0.2015 - accuracy: 0.9646 - val_loss: 0.3034 - val_accuracy: 0.9029
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 90.29217958450317
Done!
Epoch 1/100
191/191 [==============================] - 118s 546ms/step - loss: 0.5782 - accuracy: 0.6881 - val_loss: 0.4898 - val_accuracy: 0.7238
Epoch 2/10

191/191 [==============================] - 57s 300ms/step - loss: 0.2871 - accuracy: 0.9548 - val_loss: 0.4011 - val_accuracy: 0.8575
Epoch 6/100
191/191 [==============================] - 58s 303ms/step - loss: 0.2544 - accuracy: 0.9632 - val_loss: 0.3980 - val_accuracy: 0.8566
Epoch 7/100
191/191 [==============================] - 57s 301ms/step - loss: 0.2338 - accuracy: 0.9638 - val_loss: 0.4043 - val_accuracy: 0.8585
Epoch 8/100
191/191 [==============================] - 57s 300ms/step - loss: 0.2186 - accuracy: 0.9637 - val_loss: 0.3869 - val_accuracy: 0.8623
Epoch 9/100
191/191 [==============================] - 58s 301ms/step - loss: 0.2062 - accuracy: 0.9655 - val_loss: 0.3817 - val_accuracy: 0.8604
Epoch 10/100
191/191 [==============================] - 58s 304ms/step - loss: 0.1916 - accuracy: 0.9666 - val_loss: 0.3832 - val_accuracy: 0.8604
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 86.22641563415527
Done!
Epoch 1/100


191/191 [==============================] - 67s 351ms/step - loss: 0.3309 - accuracy: 0.9566 - val_loss: 0.4064 - val_accuracy: 0.8690
Epoch 7/100
191/191 [==============================] - 63s 332ms/step - loss: 0.3057 - accuracy: 0.9618 - val_loss: 0.4006 - val_accuracy: 0.8690
Epoch 8/100
191/191 [==============================] - 59s 310ms/step - loss: 0.2918 - accuracy: 0.9601 - val_loss: 0.3901 - val_accuracy: 0.8756
Epoch 9/100
191/191 [==============================] - 64s 335ms/step - loss: 0.2766 - accuracy: 0.9627 - val_loss: 0.3864 - val_accuracy: 0.8765
Epoch 10/100
191/191 [==============================] - 64s 336ms/step - loss: 0.2639 - accuracy: 0.9627 - val_loss: 0.3772 - val_accuracy: 0.8690
Epoch 11/100
191/191 [==============================] - 65s 341ms/step - loss: 0.2552 - accuracy: 0.9600 - val_loss: 0.3713 - val_accuracy: 0.8728
Epoch 12/100
191/191 [==============================] - 65s 342ms/step - loss: 0.2441 - accuracy: 0.9615 - val_loss: 0.3678 - val_accu

191/191 [==============================] - 62s 325ms/step - loss: 0.3951 - accuracy: 0.9419 - val_loss: 0.4303 - val_accuracy: 0.8822
Epoch 4/100
191/191 [==============================] - 62s 322ms/step - loss: 0.3641 - accuracy: 0.9516 - val_loss: 0.4153 - val_accuracy: 0.8841
Epoch 5/100
191/191 [==============================] - 62s 323ms/step - loss: 0.3447 - accuracy: 0.9566 - val_loss: 0.4019 - val_accuracy: 0.8878
Epoch 6/100
191/191 [==============================] - 60s 314ms/step - loss: 0.3206 - accuracy: 0.9600 - val_loss: 0.3990 - val_accuracy: 0.8737
Epoch 7/100
191/191 [==============================] - 60s 315ms/step - loss: 0.3126 - accuracy: 0.9555 - val_loss: 0.3870 - val_accuracy: 0.8812
Epoch 8/100
191/191 [==============================] - 62s 324ms/step - loss: 0.2901 - accuracy: 0.9633 - val_loss: 0.3792 - val_accuracy: 0.8803
Epoch 9/100
191/191 [==============================] - 62s 326ms/step - loss: 0.2805 - accuracy: 0.9596 - val_loss: 0.3744 - val_accurac

Epoch 2/100
191/191 [==============================] - 96s 502ms/step - loss: 0.5568 - accuracy: 0.9030 - val_loss: 0.5548 - val_accuracy: 0.8321
Epoch 3/100
191/191 [==============================] - 95s 498ms/step - loss: 0.4944 - accuracy: 0.9291 - val_loss: 0.5413 - val_accuracy: 0.8387
Epoch 4/100
191/191 [==============================] - 96s 501ms/step - loss: 0.4475 - accuracy: 0.9521 - val_loss: 0.5057 - val_accuracy: 0.8472
Epoch 5/100
191/191 [==============================] - 96s 503ms/step - loss: 0.4098 - accuracy: 0.9534 - val_loss: 0.4785 - val_accuracy: 0.8509
Epoch 6/100
191/191 [==============================] - 97s 508ms/step - loss: 0.3765 - accuracy: 0.9584 - val_loss: 0.4605 - val_accuracy: 0.8557
Epoch 7/100
191/191 [==============================] - 99s 517ms/step - loss: 0.3491 - accuracy: 0.9591 - val_loss: 0.4452 - val_accuracy: 0.8528
Epoch 8/100
191/191 [==============================] - 97s 507ms/step - loss: 0.3254 - accuracy: 0.9601 - val_loss: 0.4495 -

## Summary

In [39]:
record3.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
1,relu,2,88.030159,85.768145,88.218659,87.370408,88.124412,89.066917,88.113207,87.641507,90.094340,87.830186,88.025794
2,relu,3,86.993402,89.066917,89.066917,86.333650,88.689917,88.124412,86.603773,87.264150,88.113207,88.584906,87.884125
0,relu,1,87.276155,87.464654,88.312912,87.747407,87.276155,87.653160,87.641507,87.830186,86.792451,90.188682,87.818327
4,relu,5,86.427897,87.747407,86.522150,90.292180,87.276155,88.030159,88.490564,86.226416,87.547171,88.207549,87.676765
3,relu,4,87.841660,88.689917,88.784164,88.218659,87.087655,86.522150,87.452829,86.415094,87.075472,88.679248,87.676685
5,relu,6,85.296887,88.030159,85.862392,88.030159,89.066917,89.161170,87.830186,89.811319,86.886793,85.566038,87.554202


In [40]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_MPQA_3.xlsx', sheet_name='dynamic')